<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/PFG_RNN_Predictiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pip install unidecode

In [5]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
import unicodedata

In [6]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/RNN'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

['df_8000_rows.xlsx', 'RN_SAAF.xlsx', 'RN_AT.xlsx', 'RN_LOG.xlsx', 'RN_ACT.xlsx', 'RN_EM.xlsx', 'RN_SO.xlsx', 'PROVINCIA_MUNICIPIO.xlsx']


In [7]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

df_8000_rows.xlsx
RN_SAAF.xlsx
RN_AT.xlsx
RN_LOG.xlsx
RN_ACT.xlsx
RN_EM.xlsx
RN_SO.xlsx
PROVINCIA_MUNICIPIO.xlsx


In [12]:
RN_SAAF = dic_dataframes.get("RN_SAAF.xlsx")
RN_AT = dic_dataframes.get("RN_AT.xlsx")
RN_EM = dic_dataframes.get("RN_EM.xlsx")
RN_SO = dic_dataframes.get("RN_SO.xlsx")
RN_LOG = dic_dataframes.get("RN_LOG.xlsx")
df_8000rows = dic_dataframes.get("df_8000_rows.xlsx")
provincia_municipio = dic_dataframes.get('PROVINCIA_MUNICIPIO.xlsx')

In [13]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
# RN_SAAF = pd.read_excel('/content/RN_SAAF.xlsx')
# RN_AT = pd.read_excel('/content/RN_AT.xlsx')
# # RN_ACT = pd.read_excel('/content/RN_ACT.xlsx')
# RN_EM = pd.read_excel('/content/RN_EM.xlsx')
# RN_LOG = pd.read_excel('/content/RN_LOG.xlsx')
# RN_SO = pd.read_excel('/content/RN_SO.xlsx')
# df_8000rows = pd.read_excel('/content/df_8000_rows.xlsx')
# provincia_municipio = pd.read_excel('/content/PROVINCIA_MUNICIPIO.xlsx')

In [14]:
# Convert all string columns to uppercase and remove accents/diacritics in combined_df
df_8000rows = df_8000rows.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)
provincia_municipio = provincia_municipio.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

In [15]:
# Function to add PROVINCIA column based on LOCALIDAD
def add_provincia_column(df, provincia_mapping):
    df['PROVINCIA'] = df['LOCALIDAD'].map(provincia_mapping)
    return df

# Create a dictionary mapping from LOCALIDAD to PROVINCIA
localidad_to_provincia = provincia_municipio.set_index('MUNICIPIO')['PROVINCIA'].to_dict()

# List of RN dataframes to process
rn_dataframes = {'AT': RN_AT, 'EM': RN_EM, 'LOG': RN_LOG, 'SAAF': RN_SAAF, 'SO': RN_SO}

# Add the PROVINCIA column to each RN dataframe
rn_dfs_with_provincia = {name: add_provincia_column(df, localidad_to_provincia) for name, df in rn_dataframes.items()}

# Drop rows with null values in the 'PROVINCIA' column in each RN dataframe
rn_dfs_cleaned = {name: df.dropna(subset=['PROVINCIA']) for name, df in rn_dfs_with_provincia.items()}

In [16]:
# List of columns to keep
required_columns = ['LOCALIDAD', 'GENERO', 'YEAR NACIMIENTO', 'PROVINCIA']

# Function to ensure the dataframe has all required columns
def ensure_columns(df, columns):
    for column in columns:
        if column not in df.columns:
            df[column] = np.nan  # Create column with NaN values if it does not exist
    return df[columns]  # Return dataframe with only the required columns

# Ensure all dataframes have the required columns
rn_dfs_final = {name: ensure_columns(df, required_columns) for name, df in rn_dfs_cleaned.items()}

<ipython-input-16-605fc2068a84>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.nan  # Create column with NaN values if it does not exist


In [17]:
# Mapping dictionary for normalizing gender values
gender_mapping = {
    'FEMENINO': 'MUJER',
    'M': 'MUJER',
    'MASCULINO': 'HOMBRE',
    'H': 'HOMBRE'
}

# Function to normalize the 'GENERO' column in a dataframe
def normalize_gender_column(df):
    df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))
    return df

# Apply the normalization to the 'SO' dataframe
rn_dfs_final['SO'] = normalize_gender_column(rn_dfs_final['SO'])

<ipython-input-17-333b05a14062>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))


In [18]:
# Function to insert random registers into a dataframe where the 'GENERO' column is empty
def insert_random_registers(df, gender_distribution):
    # Find the indices where 'GENERO' is empty
    empty_indices = df[df['GENERO'].isna()].index

    # Generate random genders based on the given distribution for the empty indices
    genders = np.random.choice(
        ['MUJER', 'HOMBRE'],
        size=len(empty_indices),
        p=[gender_distribution['MUJER'], gender_distribution['HOMBRE']]
    )

    # Assign these random genders to the empty 'GENERO' rows
    df.loc[empty_indices, 'GENERO'] = genders
    return df

# Example of usage
gender_distribution = {'MUJER': 0.44, 'HOMBRE': 0.56}

# Insert random registers into a specific dataframe, e.g., 'ACT', where 'GENERO' is empty
rn_dfs_final['SAAF'] = insert_random_registers(rn_dfs_final['SAAF'], gender_distribution)
rn_dfs_final['SO'] = insert_random_registers(rn_dfs_final['SO'], gender_distribution)

In [19]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].min())
max_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    # 75% of the years under 2004
    under_2004 = np.random.randint(min_year, 2004, size=int(size * 0.75))
    # 25% of the years between 2004 and max_year
    over_2004 = np.random.randint(2004, max_year + 1, size=size - int(size * 0.75))
    return np.concatenate([under_2004, over_2004])

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))
    np.random.shuffle(random_years)  # Shuffle to mix the values

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the 'SO' dataframe
rn_dfs_final['SO'] = insert_random_years(rn_dfs_final['SO'])

In [20]:
# Add a column named 'SERVICIO' with the name of the dataframe to each dataframe in rn_dfs_final
def add_servicio_column(dfs):
    for name, df in dfs.items():
        df['SERVICIO'] = name
    return dfs

# Apply the function to rn_dfs_final
rn_dfs_final = add_servicio_column(rn_dfs_final)

<ipython-input-20-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-20-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-20-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [21]:
# Combine all dataframes into a single dataframe
combined_df = pd.concat(rn_dfs_final.values(), ignore_index=True)

# Shuffle the rows of the combined dataframe
shuffled_combined_df = combined_df.sample(frac=1).reset_index(drop=True)

shuffled_combined_df['SERVICIO'].value_counts()

SERVICIO
SO      1457
SAAF     567
EM       203
LOG      182
AT       143
Name: count, dtype: int64

In [22]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'ANTES DE 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    elif year < 2020:
        return '2000-2020'
    else:
        return '2020 EN ADELANTE'

rna1 = shuffled_combined_df.copy()
# Aplicar la función a la columna 'YEAR NACIMIENTO'
rna1['YEAR NACIMIENTO'] = rna1['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(rna1)

                 LOCALIDAD  GENERO   YEAR NACIMIENTO    PROVINCIA SERVICIO
0                   CUENCA   MUJER         2000-2020       CUENCA      LOG
1              GUADALAJARA  HOMBRE         1960-1979  GUADALAJARA       SO
2         CASAS DE BENITEZ  HOMBRE         2000-2020       CUENCA       SO
3              GUADALAJARA  HOMBRE         1960-1979  GUADALAJARA       SO
4                   RIOPAR  HOMBRE         2000-2020     ALBACETE       SO
...                    ...     ...               ...          ...      ...
2547  CABANILLAS DEL CAMPO  HOMBRE         2000-2020  GUADALAJARA     SAAF
2548           GUADALAJARA   MUJER         1960-1979  GUADALAJARA      LOG
2549              ALBACETE  HOMBRE         1940-1959     ALBACETE       SO
2550           MIGUELTURRA   MUJER  2020 EN ADELANTE  CIUDAD REAL       EM
2551           CIUDAD REAL   MUJER         1980-1999  CIUDAD REAL     SAAF

[2552 rows x 5 columns]


##RNA predictiva con los datos reales
Vamos a realizar una primera red neuronal con los registros reales.

####Preparación de los datos

Convertimos las variables categóricas en variables dummy usando pd.get_dummies.

Separamos las características (x) de la etiqueta (y).
Convertimos la etiqueta en variables dummy para la clasificación multiclase.

In [23]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(rna1, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

####División de los datos
Usaremos un 70% de los datos para entrenar y del 30% restante, un 70% para validar (21% del total) e ir ajustandio los hiperparámetros (evitando el sobreentrenamiento) y un 30% para el conjunto de test (9% del total), para evaluar el modelo y su precisión al final.

In [24]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (1786, 228)
Tamaño del conjunto de validación: (536, 228)
Tamaño del conjunto de prueba: (230, 228)


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.3951 - loss: 1.4931 - val_accuracy: 0.5653 - val_loss: 1.1705
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5719 - loss: 1.1719 - val_accuracy: 0.5802 - val_loss: 1.0915
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6028 - loss: 1.0551 - val_accuracy: 0.5970 - val_loss: 1.0463
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6078 - loss: 1.0755 - val_accuracy: 0.6399 - val_loss: 1.0101
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6294 - loss: 0.9999 - val_accuracy: 0.6437 - val_loss: 0.9872
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6193 - loss: 0.9742 - val_accuracy: 0.6437 - val_loss: 0.9550
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6502 - loss: 0.9133 - val_accuracy: 0.6530 - val_loss: 0.9346
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6722 - loss: 0.8825 - val_accuracy: 0.6567 - val_loss

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu, tanh

# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3579 - loss: 1.5183 - val_accuracy: 0.5653 - val_loss: 1.1766
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5938 - loss: 1.1378 - val_accuracy: 0.5784 - val_loss: 1.0711
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6016 - loss: 1.0555 - val_accuracy: 0.6119 - val_loss: 1.0226
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6130 - loss: 1.0294 - val_accuracy: 0.6343 - val_loss: 0.9865
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6352 - loss: 0.9843 - val_accuracy: 0.6437 - val_loss: 0.9554
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6817 - loss: 0.9153 - val_accuracy: 0.6511 - val_loss: 0.9398
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6545 - loss: 0.9047 - val_accuracy: 0.6511 - val_loss: 0.9279
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6799 - loss: 0.8762 - val_accuracy: 0.6511 - val_loss: 0.

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3333 - loss: 1.5016 - val_accuracy: 0.5653 - val_loss: 1.1721
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5721 - loss: 1.1624 - val_accuracy: 0.5653 - val_loss: 1.1007
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5733 - loss: 1.0690 - val_accuracy: 0.5821 - val_loss: 1.0672
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5868 - loss: 1.0437 - val_accuracy: 0.5951 - val_loss: 1.0437
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6105 - loss: 1.0005 - val_accuracy: 0.6493 - val_loss: 1.0120
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6328 - loss: 0.9730 - val_accuracy: 0.6511 - val_loss: 0.9953
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6219 - loss: 0.9492 - val_accuracy: 0.6511 - val_loss: 0.9823
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6652 - loss: 0.8883 - val_accuracy: 0.6455 - val_loss: 0.9620
Epo

###Modelo XGBoost

In [28]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Codificar las etiquetas usando LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.idxmax(axis=1))  # Convertir de one-hot encoding a etiquetas enteras

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Crear y entrenar el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))
xgb_model.fit(X_train_xgb, y_train_xgb)

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = xgb_model.predict_proba(X_train_xgb)
X_test_nn = xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Definir el modelo de red neuronal
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1])

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4403 - loss: 1.5388 - val_accuracy: 0.5894 - val_loss: 1.2307
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5654 - loss: 1.1605 - val_accuracy: 0.6983 - val_loss: 0.9252
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6594 - loss: 0.9227 - val_accuracy: 0.7039 - val_loss: 0.8431
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6678 - loss: 0.8850 - val_accuracy: 0.7011 - val_loss: 0.8172
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6906 - loss: 0.8109 - val_accuracy: 0.7011 - val_loss: 0.7913
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6811 - loss: 0.8398 - val_accuracy: 0.7263 - val_loss: 0.7779
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7041 - loss: 0.7851 - val_accuracy: 0.7039 - val_loss: 0.7655
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7287 - loss: 0.7538 - val_accuracy: 0.7179 - val_loss: 0.

In [29]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Original Servicio: SAAF
Servicio: AT, Probabilidad: 0.0033
Servicio: EM, Probabilidad: 0.0063
Servicio: LOG, Probabilidad: 0.0026
Servicio: SAAF, Probabilidad: 0.5044
Servicio: SO, Probabilidad: 0.4834


In [30]:
import numpy as np

# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[22]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[22]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.0010
Servicio: EM, Probabilidad: 0.0036
Servicio: LOG, Probabilidad: 0.0196
Servicio: SAAF, Probabilidad: 0.1712
Servicio: SO, Probabilidad: 0.8046


In [31]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[70]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[70]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Original Servicio: SAAF
Servicio: AT, Probabilidad: 0.4120
Servicio: EM, Probabilidad: 0.0330
Servicio: LOG, Probabilidad: 0.0115
Servicio: SAAF, Probabilidad: 0.2563
Servicio: SO, Probabilidad: 0.2872


##RNA predictiva con datos generados
Red Neuronal con 8000 registros más que han sido creados teniendo en cuenta las estadísticas reales de los datos, es decir, contemplando cómo se distribuyen por edad, género y localidad y, obviamente, el municipio y la provincia correctamente computados.

Esto se hace para intentar aumentar el porcentaje de precisión obtenido.

In [32]:
# Calculate the frequency of each element in the 'SERVICIO' column
service_frequency = df['SERVICIO'].value_counts()

# Group by 'PROVINCIA' and 'SERVICIO' and count the occurrences
service_province_freq = shuffled_combined_df.groupby(['PROVINCIA', 'SERVICIO']).size().reset_index(name='COUNT')

# Function to assign services based on frequency
def assign_service(row, freq_df):
    province = row['PROVINCIA']
    possible_services = freq_df[freq_df['PROVINCIA'] == province]
    if possible_services.empty:
        return np.nan
    else:
        return np.random.choice(possible_services['SERVICIO'].values, p=possible_services['COUNT'].values / possible_services['COUNT'].sum())

# Apply the function to assign services
df_8000rows['SERVICIO'] = df_8000rows.apply(assign_service, axis=1, args=(service_province_freq,))

# Ensure balanced classes by sampling
min_class_size = service_frequency.min()
balanced_df = df_8000rows.groupby('SERVICIO').apply(lambda x: x.sample(n=min_class_size, replace=True)).reset_index(drop=True)

In [33]:
# Calculate the frequency of each service
service_frequency = shuffled_combined_df['SERVICIO'].value_counts()

# Set the target number of records per service
target_size = 1457

# Create a balanced dataframe starting with the initial shuffled data
to_use_df = shuffled_combined_df.copy()

# Loop through each service to balance the dataframe
for service, count in service_frequency.items():
    if count < target_size:
        # Get the rows for the current service
        service_df = balanced_df[balanced_df['SERVICIO'] == service]
        # Sample additional rows with replacement to reach the target size
        additional_samples = service_df.sample(n=target_size - count, replace=True)
        to_use_df = pd.concat([to_use_df, additional_samples])

# Shuffle the final balanced dataframe
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [34]:
# Shuffle the rows of the dataframe ready to use
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [35]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(to_use_df['YEAR NACIMIENTO'].min())
max_year = int(to_use_df['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    return np.random.randint(min_year, max_year + 1, size=int(size))

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the dataframe
to_use_df = insert_random_years(to_use_df)

In [36]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'Antes de 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    else:
        return '2000 en adelante'

# Aplicar la función a la columna 'YEAR NACIMIENTO'
to_use_df['YEAR NACIMIENTO'] = to_use_df['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(to_use_df.head())

             LOCALIDAD  GENERO   YEAR NACIMIENTO    PROVINCIA SERVICIO  \
0             CAMARENA   MUJER  2000 en adelante       TOLEDO     SAAF   
1  AZUQUECA DE HENARES  HOMBRE  2000 en adelante  GUADALAJARA       SO   
2               CUENCA   MUJER  2000 en adelante       CUENCA     SAAF   
3             FONTANAR   MUJER  2000 en adelante  GUADALAJARA       SO   
4            FUENLLANA   MUJER         1940-1959  CIUDAD REAL       EM   

  TIPO SORDERA  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4        MIXTA  


In [37]:
# Determina la cantidad de servicios únicos
unique_services = to_use_df['SERVICIO'].unique()
num_services = len(unique_services)

# Determina cuántos registros por servicio necesitas para el conjunto de prueba
test_size_per_service = 85 // num_services

# Si hay más de 85 registros, ajusta la muestra para que esté balanceada
if test_size_per_service * num_services < 85:
    additional_samples = 85 - (test_size_per_service * num_services)
else:
    additional_samples = 0

# Crear el conjunto de prueba balanceado
test_df = pd.DataFrame()

for service in unique_services:
    service_df = to_use_df[to_use_df['SERVICIO'] == service]
    test_samples = service_df.sample(n=test_size_per_service, replace=False)
    test_df = pd.concat([test_df, test_samples])

# Añadir muestras adicionales si es necesario
if additional_samples > 0:
    additional_df = to_use_df[~to_use_df.index.isin(test_df.index)]
    additional_samples_df = additional_df.sample(n=additional_samples, replace=False)
    test_df = pd.concat([test_df, additional_samples_df])

# Eliminar las filas seleccionadas del dataframe original para crear el conjunto de entrenamiento/validación
to_use_df = to_use_df.drop(test_df.index)

# Verifica el conjunto de prueba
print(test_df['SERVICIO'].value_counts())
print(test_df.shape)

# Verifica el conjunto de entrenamiento/validación
print(to_use_df.shape)

SERVICIO
SAAF    17
SO      17
EM      17
AT      17
LOG     17
Name: count, dtype: int64
(85, 6)
(7200, 6)


In [38]:
to_use_df = to_use_df.drop(columns=['TIPO SORDERA'])

#Red Neuronal Predictiva

Creación de una red neuronal predictiva que sea capaz de clasificar nuevos usuarios con buena precisión en servicio y provincia.

In [40]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(to_use_df, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

In [41]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (5040, 520)
Tamaño del conjunto de validación: (1512, 520)
Tamaño del conjunto de prueba: (648, 520)


In [42]:
# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.2851 - loss: 1.5700 - val_accuracy: 0.5033 - val_loss: 1.3135
Epoch 2/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4421 - loss: 1.3016 - val_accuracy: 0.6091 - val_loss: 1.0315
Epoch 3/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5822 - loss: 1.0581 - val_accuracy: 0.7507 - val_loss: 0.7801
Epoch 4/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7046 - loss: 0.8360 - val_accuracy: 0.7851 - val_loss: 0.6170
Epoch 5/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7577 - loss: 0.6828 - val_accuracy: 0.8062 - val_loss: 0.5492
Epoch 6/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7913 - loss: 0.5948 - val_accuracy: 0.8062 - val_loss: 0.5060
Epoch 7/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7938 - loss: 0.5634 - val_accuracy: 0.8214 - val_loss: 0.4816
Epoch 8/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8087 - loss: 0.5157 - val_accuracy: 0.8294 - val_

In [43]:
# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.2889 - loss: 1.5622 - val_accuracy: 0.4828 - val_loss: 1.2998
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4828 - loss: 1.2557 - val_accuracy: 0.6402 - val_loss: 1.0536
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6400 - loss: 1.0168 - val_accuracy: 0.7183 - val_loss: 0.8432
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7231 - loss: 0.8056 - val_accuracy: 0.7374 - val_loss: 0.7117
Epoch 5/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7537 - loss: 0.6802 - val_accuracy: 0.7440 - val_loss: 0.6574
Epoch 6/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7538 - loss: 0.6261 - val_accuracy: 0.7487 - val_loss: 0.6348
Epoch 7/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7658 - loss: 0.6028 - val_accuracy: 0.7507 - val_loss: 0.6270
Epoch 8/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7564 - loss: 0.5905 - val_accuracy: 0.7606 - val_loss: 0.

In [44]:
# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2546 - loss: 1.5816 - val_accuracy: 0.4034 - val_loss: 1.3688
Epoch 2/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4306 - loss: 1.3239 - val_accuracy: 0.6184 - val_loss: 1.0322
Epoch 3/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6230 - loss: 1.0151 - val_accuracy: 0.7612 - val_loss: 0.7377
Epoch 4/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7431 - loss: 0.7774 - val_accuracy: 0.7877 - val_loss: 0.6162
Epoch 5/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7660 - loss: 0.6552 - val_accuracy: 0.8115 - val_loss: 0.5587
Epoch 6/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7874 - loss: 0.5987 - val_accuracy: 0.8148 - val_loss: 0.5354
Epoch 7/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8004 - loss: 0.5410 - val_accuracy: 0.8221 - val_loss: 0.5059
Epoch 8/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8189 - loss: 0.5054 - val_accuracy: 0.8320 - val_

Ejemplo de prueba para ver qué tan bien predice.

In [45]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[3]

# Obtener el servicio objetivo original
original_service = y_test.iloc[3].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.0001
Servicio: EM, Probabilidad: 0.9977
Servicio: LOG, Probabilidad: 0.0000
Servicio: SAAF, Probabilidad: 0.0000
Servicio: SO, Probabilidad: 0.0021


In [46]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[22]

# Obtener el servicio objetivo original
original_service = y_test.iloc[22].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Original Servicio: SAAF
Servicio: AT, Probabilidad: 0.0019
Servicio: EM, Probabilidad: 0.0007
Servicio: LOG, Probabilidad: 0.0002
Servicio: SAAF, Probabilidad: 0.9928
Servicio: SO, Probabilidad: 0.0044


In [47]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[70]

# Obtener el servicio objetivo original
original_service = y_test.iloc[70].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.0020
Servicio: EM, Probabilidad: 0.0020
Servicio: LOG, Probabilidad: 0.1820
Servicio: SAAF, Probabilidad: 0.1800
Servicio: SO, Probabilidad: 0.6341


In [48]:
pip install scikeras

In [49]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Definir el modelo con hiperparámetros ajustables
def create_nn_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el KerasClassifier con SciKeras
nn_model = KerasClassifier(model=create_nn_model, verbose=0)

# Definir los hiperparámetros para la búsqueda aleatoria
param_dist_nn = {
    'model__learning_rate': uniform(0.0001, 0.001),
    'model__dropout_rate': uniform(0.1, 0.5),
    'epochs': [50, 100, 200],
    'batch_size': [16, 32, 64]
}

# Realizar la búsqueda aleatoria
random_search_nn = RandomizedSearchCV(estimator=nn_model, param_distributions=param_dist_nn, scoring='accuracy', cv=3, n_iter=20, verbose=1, random_state=42)
random_search_nn.fit(X_train, y_train)

# Obtener el mejor modelo
best_nn_model = random_search_nn.best_estimator_

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

TypeError: cannot unpack non-iterable float object

In [51]:
# Evaluar el mejor modelo de red neuronal
best_nn_model_keras = best_nn_model.model_
loss, accuracy = best_nn_model_keras.evaluate(X_test, y_test)
print(f'Best Neural Network Model Loss: {loss}')
print(f'Best Neural Network Model Accuracy: {accuracy}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8084 - loss: 0.6478
Best Neural Network Model Loss: 0.5997592806816101
Best Neural Network Model Accuracy: 0.8302469253540039


##Modelo XGBoost con datos generados

In [52]:
# Codificar las etiquetas usando LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.idxmax(axis=1))  # Convertir de one-hot encoding a etiquetas enteras

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Crear y entrenar el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))
xgb_model.fit(X_train_xgb, y_train_xgb)

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = xgb_model.predict_proba(X_train_xgb)
X_test_nn = xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Definir el modelo de red neuronal
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1])

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.5397 - loss: 1.4476 - val_accuracy: 0.7808 - val_loss: 0.6637
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7899 - loss: 0.6611 - val_accuracy: 0.7927 - val_loss: 0.5445
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7898 - loss: 0.5730 - val_accuracy: 0.7986 - val_loss: 0.5378
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8096 - loss: 0.5466 - val_accuracy: 0.7946 - val_loss: 0.5412
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8083 - loss: 0.5380 - val_accuracy: 0.7986 - val_loss: 0.5398
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8127 - loss: 0.5344 - val_accuracy: 0.7946 - val_loss: 0.5290
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8089 - loss: 0.5298 - val_accuracy: 0.7976 - val_loss: 0.5229
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8014 - loss: 0.5221 - val_accuracy: 0.7927 - val_lo

In [53]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Original Servicio: LOG
Servicio: AT, Probabilidad: 0.3839
Servicio: EM, Probabilidad: 0.0230
Servicio: LOG, Probabilidad: 0.0352
Servicio: SAAF, Probabilidad: 0.5219
Servicio: SO, Probabilidad: 0.0360


In [54]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Original Servicio: LOG
Servicio: AT, Probabilidad: 0.3839
Servicio: EM, Probabilidad: 0.0230
Servicio: LOG, Probabilidad: 0.0352
Servicio: SAAF, Probabilidad: 0.5219
Servicio: SO, Probabilidad: 0.0360


In [55]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Original Servicio: LOG
Servicio: AT, Probabilidad: 0.3839
Servicio: EM, Probabilidad: 0.0230
Servicio: LOG, Probabilidad: 0.0352
Servicio: SAAF, Probabilidad: 0.5219
Servicio: SO, Probabilidad: 0.0360


In [59]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Definir el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))

# Definir los hiperparámetros para la búsqueda en cuadrícula
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Realizar la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)
grid_search.fit(X_train_xgb, y_train_xgb)

# Obtener el mejor modelo
best_xgb_model = grid_search.best_estimator_

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = best_xgb_model.predict_proba(X_train_xgb)
X_test_nn = best_xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Evaluar el mejor modelo XGBoost
best_xgb_accuracy = grid_search.best_score_
print(f'Best XGBoost Model Accuracy: {best_xgb_accuracy}')

Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best XGBoost Model Accuracy: 0.736904761904762


In [57]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Definir el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))

# Definir los hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

# Realizar la búsqueda aleatoria
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, scoring='accuracy', cv=3, n_iter=50, verbose=1, random_state=42)
random_search.fit(X_train_xgb, y_train_xgb)

# Obtener el mejor modelo
best_xgb_model = random_search.best_estimator_

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = best_xgb_model.predict_proba(X_train_xgb)
X_test_nn = best_xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Evaluar el mejor modelo XGBoost
best_xgb_accuracy = random_search.best_score_
print(f'Best XGBoost Model Accuracy: {best_xgb_accuracy}')

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best XGBoost Model Accuracy: 0.781547619047619


In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Definir el modelo de red neuronal con regularización y Dropout
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3, regularization=0.01):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1], learning_rate=0.001, dropout_rate=0.4, regularization=0.01)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.3870 - loss: 2.6331 - val_accuracy: 0.8075 - val_loss: 1.7463
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7673 - loss: 1.5855 - val_accuracy: 0.8165 - val_loss: 1.1083
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8155 - loss: 1.1004 - val_accuracy: 0.8145 - val_loss: 0.9426
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8104 - loss: 0.9750 - val_accuracy: 0.8155 - val_loss: 0.8840
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8234 - loss: 0.8881 - val_accuracy: 0.8204 - val_loss: 0.8334
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8144 - loss: 0.8767 - val_accuracy: 0.8254 - val_loss: 0.8099
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8235 - loss: 0.8433 - val_accuracy: 0.8214 - val_loss: 0.7932
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8189 - loss: 0.8277 - val_accuracy: 0.8254 - val_loss: 0.

In [62]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[2]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = best_xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[2]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Original Servicio: AT
Servicio: AT, Probabilidad: 0.9766
Servicio: EM, Probabilidad: 0.0030
Servicio: LOG, Probabilidad: 0.0018
Servicio: SAAF, Probabilidad: 0.0089
Servicio: SO, Probabilidad: 0.0098
